In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

In [65]:
# Load the data from the data folder
wind_forecast = pd.read_csv('../Data/1.Wind forecast profile.csv', delimiter=';')
load = pd.read_csv('../Data/1.Load profile.csv', delimiter=';')
bus = pd.read_csv('../Data/B (power transfer factor of each bus to each line).csv', delimiter=';')
max_prod = pd.read_csv('../Data/Maximum production of generating units.csv', delimiter=';')
min_prod = pd.read_csv('../Data/Minimum production of generating units.csv', delimiter=';')
min_down_time = pd.read_csv('../Data/Minimum down time of generating units.csv', delimiter=';')
min_up_time = pd.read_csv('../Data/Minimum up time of generating units.csv', delimiter=';')
prod_cost = pd.read_csv('../Data/Production cost of generating units.csv', delimiter=';')
ramp_rate = pd.read_csv('../Data/Ramping rate of generating units.csv', delimiter=';')
start_up_cost = pd.read_csv('../Data/Start-up cost of generating units.csv', delimiter=';')
transmission_cap = pd.read_csv('../Data/Transmission capacity of lines.csv', delimiter=';')


In [66]:
Nodes = ['Node 1', 'Node 2', 'Node 3', 'Node 4', 'Node 5', 'Node 6']
Generator = ['G1', 'G2', 'G3']
Generator_node = {'Node 1': 'G1', 'Node 2': 'G2', 'Node 6': 'G3'}
Load = ['L1', 'L2', 'L3']
Load_node = {'Node 4': 'L1', 'Node 5': 'L2', 'Node 6': 'L3'}
Wind = ['W1', 'W2']
Wind_node = {'Node 4': 'W1', 'Node 5': 'W2'}

In [67]:
# Create matrix with the nodes as columns and the generators, loads and winds as rows, with 1 if connected to the node
Gen_n = np.zeros((len(Generator), len(Nodes)))
Load_n = np.zeros((len(Load), len(Nodes)))
Wind_n = np.zeros((len(Wind), len(Nodes)))

# Populate the matrix
for i, g in enumerate(Generator):  # Iterate over generators
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Generator_node.get(node) == g:  # Check if generator is connected to the node
            Gen_n[i, j] = 1

for i, l in enumerate(Load):  # Iterate over loads
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Load_node.get(node) == l:  # Check if load is connected to the node
            Load_n[i, j] = 1

for i, w in enumerate(Wind):  # Iterate over winds
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Wind_node.get(node) == w:  # Check if wind is connected to the node
            Wind_n[i, j] = 1
            


In [68]:
# Define the input data class
class InputData:
    
    def __init__(
        self,
        wind_forecast: pd.DataFrame, 
        bus: pd.DataFrame,
        load: pd.DataFrame,
        max_prod: pd.DataFrame,
        min_prod: pd.DataFrame,
        min_down_time: pd.DataFrame,
        min_up_time: pd.DataFrame,
        prod_cost: pd.DataFrame,
        ramp_rate: pd.DataFrame,
        start_up_cost: pd.DataFrame,
        transmission_cap: pd.DataFrame
    ):
        self.time = wind_forecast['Hour']  #maybe define it with lenght of wind_production
        self.wind_forecast = wind_forecast
        self.bus = bus
        self.load = load
        self.max_prod = max_prod
        self.min_prod = min_prod
        self.min_down_time = min_down_time
        self.min_up_time = min_up_time
        self.prod_cost = prod_cost
        self.ramp_rate = ramp_rate
        self.start_up_cost = start_up_cost
        self.transmission_cap = transmission_cap
        self.M = 1000000  # Penalty for having flexible demand
        self.Gen_n = Gen_n  # Matrix mapping generators to nodes
        self.Load_n = Load_n # Matrix mapping loads to nodes
        self.Wind_n = Wind_n # Matrix mapping wind to nodes
        
        


In [69]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [73]:
# Define the optimization model class

class EconomicDispatch():
        
        def __init__(self, input_data: InputData):
            self.data = input_data 
            self.variables = Expando()
            self.constraints = Expando() 
            self.results = Expando() 
            self._build_model() 
            
        def _build_variables(self):
            # one binary variable for the status of each generator
            self.variables.status = {
                (i, t): self.model.addVar(vtype=GRB.BINARY, 
                                            name='status_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # one variable for each generator for each time of the day
            self.variables.prod_gen = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.max_prod.iloc[i-1, 0], 
                                           name='generation_G{}_{}'.format(i, t)) 
                                           for i in range(1, len(self.data.max_prod)+1) 
                                           for t in self.data.time}
            
            # one variable for each wind generator for each time of the day
            self.variables.prod_wind = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.wind_forecast.iloc[t, i], 
                                            name='wind_generation_W{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.wind_forecast.iloc[0, :])) 
                                            for t in self.data.time}
            
            # one variable for each start-up cost for each generator
            self.variables.start_up_cost = {
                 (i, t): self.model.addVar(lb=0, 
                                            name='start_up_cost_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # add two slack variables to always make the model feasible, allowing the demand to be flexible
            self.variables.epsilon = {
                 (n, t): self.model.addVar(lb=0, 
                                           name='epsilon_Bus{}_{}'.format(n, t)) 
                                           for n in range(1, len(self.data.bus.iloc[0,:])+1) 
                                           for t in self.data.time}
            self.variables.delta = {
                 (n, t): self.model.addVar(lb=0, 
                                           name='delta_Bus{}_{}'.format(n, t))
                                           for n in range(1, len(self.data.bus.iloc[0,:])+1)
                                           for t in self.data.time}
            
            
        def _build_constraints(self):
            # Minimum capacity of the generator
            self.constraints.min_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] >= self.data.min_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}
            # Maximum capacity of the generator
            self.constraints.max_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] <= self.data.max_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}

            # Power balance constraint
            self.constraints.power_balance = {
                t: self.model.addConstr(
                    gp.quicksum(self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod) + 1)) +
                    gp.quicksum(self.variables.prod_wind[i, t] for i in range(1, len(self.data.wind_forecast.iloc[0, :]))) == 
                    gp.quicksum(self.data.load.iloc[t, i] * Load_n[i-1, n-1] for i in range(1, len(self.data.load.iloc[0, :]))for n in range(1, len(self.data.bus.iloc[0, :]) + 1))
                    + gp.quicksum(self.variables.epsilon[n, t] - self.variables.delta[n, t] for n in range(1, len(self.data.bus.iloc[0, :]) + 1))
                ) for t in self.data.time}
            


            # # Transmission capacity constraint
            # self.constraints.transmission_capacity_up = {
            #         (l, t): self.model.addConstr(
            #             gp.quicksum(self.data.bus.iloc[l-1, n-1] *
            #                     (self.variables.prod_gen[i, t] * Gen_n[i-1, n-1] +
            #                     self.variables.prod_wind[i, t] * Wind_n[i-1, n-1] -
            #                     self.variables.epsilon[n, t] +
            #                     self.variables.delta[n, t]) for n in range(1, len(self.data.bus.iloc[0,:])+1)) <=
            #             self.data.transmission_cap.iloc[l-1, 0]
            #         ) for l in range(1, len(self.data.transmission_cap)+1)
            #         for t in self.data.time}
            


            self.constraints.transmission_capacity_up = {
                    (l, t): self.model.addConstr(
                        gp.quicksum(self.data.bus.iloc[l-1, n-1] *
                                (self.variables.prod_gen[g, t] * Gen_n[g-1, n-1] +
                                self.variables.prod_wind[w, t] * Wind_n[w-1, n-1] -
                                self.data.load.iloc[t,i] * Load_n[i-1, n-1] -
                                self.variables.epsilon[n, t] +
                                self.variables.delta[n, t]) 
                                for n in range(1, len(self.data.bus.iloc[0,:])+1)
                                for i in range (1, len(self.data.load.iloc[0, :]))
                                for g in range(1, len(self.data.max_prod)+1)
                                for w in range(1, len(self.data.wind_forecast.iloc[0, :]))) <=
                        self.data.transmission_cap.iloc[l-1, 0]
                    ) for l in range(1, len(self.data.transmission_cap)+1)
                    for t in self.data.time}
            
            self.constraints.transmission_capacity_down = {
                    (l, t): self.model.addConstr(
                        gp.quicksum(self.data.bus.iloc[l-1, n-1] *
                                (self.variables.prod_gen[g, t] * Gen_n[g-1, n-1] +
                                self.variables.prod_wind[w, t] * Wind_n[w-1, n-1] -
                                self.data.load.iloc[t,i] * Load_n[i-1, n-1] -
                                self.variables.epsilon[n, t] +
                                self.variables.delta[n, t]) 
                                for n in range(1, len(self.data.bus.iloc[0,:])+1)
                                for i in range (1, len(self.data.load.iloc[0, :]))
                                for g in range(1, len(self.data.max_prod)+1)
                                for w in range(1, len(self.data.wind_forecast.iloc[0, :]))) <=
                        -self.data.transmission_cap.iloc[l-1, 0]
                    ) for l in range(1, len(self.data.transmission_cap)+1)
                    for t in self.data.time}
            
            










            # self.constraints.transmission_capacity_down = {
            #         (l, t): self.model.addConstr(
            #             gp.quicksum(self.data.bus.iloc[l-1, n-1] *
            #                     (self.variables.prod_gen[i, t] * Gen_n[i-1, n-1] +
            #                     self.variables.prod_wind[i, t] * Wind_n[i-1, n-1] -
            #                     self.variables.epsilon[n, t] +
            #                     self.variables.delta[n, t]) for n in range(1, len(self.data.bus.iloc[0,:])+1)) >=
            #             -self.data.transmission_cap.iloc[l-1, 0]
            #         ) for l in range(1, len(self.data.transmission_cap)+1)
            #         for t in self.data.time}
                                     




            #Start-up costs constraint
            self.constraints.start_up_cost = {
                (i, t): self.model.addConstr(
                    self.variables.start_up_cost[i, t] >= self.data.start_up_cost.iloc[i-1, 0] * (self.variables.status[i, t] - self.variables.status[i, t-1])
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.start_up_cost_0 = {
                i: self.model.addConstr(
                    self.variables.start_up_cost[i, 0] >= self.data.start_up_cost.iloc[i-1, 0] * self.variables.status[i, 0]
                ) for i in range(1, len(self.data.max_prod)+1)}
            
            # Ramping constraint
            self.constraints.ramping_up = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] - self.variables.prod_gen[i, t-1] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.ramping_down = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t-1] - self.variables.prod_gen[i, t] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            
            # Minimum up time constraint
            self.constraints.min_up_time = {
                (i, t, to): self.model.addConstr(
                    -self.variables.status[i, t - 1] + self.variables.status[i, t] - self.variables.status[i, to] <= 0
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_up_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            
            # Minimum down time constraint
            self.constraints.min_down_time = {
                (i, t, to): self.model.addConstr(
                    self.variables.status[i, t - 1] - self.variables.status[i, t] + self.variables.status[i, to] <= 1
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_down_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            


        def _build_objective(self):
            # Objective function
            self.model.setObjective(
                gp.quicksum(self.data.prod_cost.iloc[i-1, 0]*self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                gp.quicksum(self.variables.start_up_cost[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                self.data.M * (gp.quicksum(self.variables.epsilon[n, t] + self.variables.delta[n, t] for n in range(1, len(self.data.bus.iloc[0,:])+1) for t in self.data.time))
            )

        def _build_model(self):
            self.model = gp.Model('EconomicDispatch')
            self._build_variables()
            self._build_constraints()
            self._build_objective()
            self.model.update()

        def optimize(self):
            self.model.optimize()
            self._extract_results()

        def _extract_results(self):
    
            self.results.production = pd.DataFrame({
                #'time': [t for t in self.data.time],
                #'status 1': [self.variables.status[1, t].x for t in self.data.time],
                #'status 2': [self.variables.status[2, t].x for t in self.data.time],
                #'status 3': [self.variables.status[3, t].x for t in self.data.time],
                #'start_up_cost 1': [self.variables.start_up_cost[1, t].x for t in self.data.time],
                #'start_up_cost 2': [self.variables.start_up_cost[2, t].x for t in self.data.time],
                #'start_up_cost 3': [self.variables.start_up_cost[3, t].x for t in self.data.time],
                
                'generation 1': [self.variables.prod_gen[1, t].x for t in self.data.time],
                'generation 2': [self.variables.prod_gen[2, t].x for t in self.data.time],
                'generation 3': [self.variables.prod_gen[3, t].x for t in self.data.time],
                'wind generation 1': [self.variables.prod_wind[1, t].x for t in self.data.time],
                'wind generation 2': [self.variables.prod_wind[2, t].x for t in self.data.time],
                'load 1': [self.data.load.iloc[t, 1] for t in self.data.time],
                'load 2': [self.data.load.iloc[t, 2] for t in self.data.time],
                'load 3': [self.data.load.iloc[t, 3] for t in self.data.time],
                'epsilon 1': [self.variables.epsilon[1, t].x for t in self.data.time],
                'delta 1': [self.variables.delta[1, t].x for t in self.data.time],
                'epsilon 2': [self.variables.epsilon[2, t].x for t in self.data.time],
                'delta 2': [self.variables.delta[2, t].x for t in self.data.time],
                'epsilon 3': [self.variables.epsilon[3, t].x for t in self.data.time],
                'delta 3': [self.variables.delta[3, t].x for t in self.data.time],
                'epsilon 4': [self.variables.epsilon[4, t].x for t in self.data.time],
                'delta 4': [self.variables.delta[4, t].x for t in self.data.time],
                'epsilon 5': [self.variables.epsilon[5, t].x for t in self.data.time],
                'delta 5': [self.variables.delta[5, t].x for t in self.data.time],
                'epsilon 6': [self.variables.epsilon[6, t].x for t in self.data.time],
                'delta 6': [self.variables.delta[6, t].x for t in self.data.time],
                'Transmission Line 1': [gp.quicksum(self.data.bus.iloc[1-1, n-1] *
                                (self.variables.prod_gen[g, t].x * Gen_n[g-1, n-1] +
                                self.variables.prod_wind[w, t].x * Wind_n[w-1, n-1] -
                                self.data.load.iloc[t,i] * Load_n[i-1, n-1] -
                                self.variables.epsilon[n, t].x +
                                self.variables.delta[n, t].x) 
                                for n in range(1, len(self.data.bus.iloc[0,:])+1)
                                for i in range (1, len(self.data.load.iloc[0, :]))
                                for g in range(1, len(self.data.max_prod)+1)
                                for w in range(1, len(self.data.wind_forecast.iloc[0, :])))
                                for t in self.data.time]
            })
            
        def _print_model(self):
            self.model.write('EconomicDispatch.lp')        
            
                 

In [74]:
# Run the model
input_data = InputData(wind_forecast, bus, load, max_prod, min_prod, min_down_time, min_up_time, prod_cost, ramp_rate, start_up_cost, transmission_cap)
model = EconomicDispatch(input_data)
model.optimize()
model.results.production
#model._build_model()
#model._print_model()


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 413 rows, 230 columns and 2694 nonzeros
Model fingerprint: 0x55b1c1e7
Variable types: 200 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [4e-01, 9e+02]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+03]
Presolve removed 178 rows and 21 columns
Presolve time: 0.00s
Presolved: 235 rows, 209 columns, 1555 nonzeros
Variable types: 170 continuous, 39 integer (39 binary)
Found heuristic solution: objective 1.673050e+09

Root relaxation: objective 1.552460e+09, 130 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

,generation 1,generation 2,generation 3,wind generation 1,wind generation 2,load 1,load 2,load 3,epsilon 1,delta 1,...,delta 2,epsilon 3,delta 3,epsilon 4,delta 4,epsilon 5,delta 5,epsilon 6,delta 6,Transmission Line 1
0,139.538329,0.0,70.0,5.000000,5.0,1,200,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,30.628338,0.0,64.166667,0.0,86.666667,-780.0000010257255
1,137.371662,0.0,70.0,10.000000,7.0,3,200,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,28.795004,0.0,63.166667,0.0,86.666667,-780.0000010257255
2,139.704996,0.0,70.0,7.000000,8.0,5,200,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,30.961671,0.0,62.666667,0.0,86.666667,-780.0000010257255
3,100.000000,0.0,70.0,23.000000,10.0,0,100,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,5.687176,0.0,28.333333,0.0,86.666667,-353.63084076945074
4,100.000000,0.0,20.0,23.126615,2.0,0,10,20,128.57106,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,2.333333,0.0,43.333333,-200.00000000000023
5,100.000000,0.0,70.0,15.000000,5.0,0,100,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,12.268951,0.0,30.833333,0.0,86.666667,-424.1588781707744
6,100.000000,0.0,70.0,20.000000,7.0,0,100,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,8.378764,0.0,29.833333,0.0,86.666667,-386.1822426469853
7,100.000000,0.0,70.0,13.000000,8.0,0,100,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,13.070353,0.0,29.333333,0.0,86.666667,-418.7336445245184
8,100.000000,0.0,70.0,23.000000,10.0,0,100,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,5.687176,0.0,28.333333,0.0,86.666667,-353.63084076945074
9,100.000000,0.0,70.0,24.000000,2.0,0,100,200,0.00000,0.0,...,0.0,0.0,0.0,0.0,6.577362,0.0,32.333333,0.0,86.666667,-391.6074762932405
